In [ ]:
import torch
from tqdm import tqdm
from network.conv_node import NODE
from misc import *
import os
from pathlib import Path
import numpy as np
import pickle

# GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model = NODE(device, (3, 400, 600), 32, augment_dim=0, time_dependent=True, adjoint=True)
model.eval()
model.to(device)
model.load_state_dict(torch.load(f'/home/lbw/CLODE/pth/pth/lowlight.pth', weights_only=True), strict=False)

# 결과 저장을 위한 디렉토리 생성
cache_dir = Path('/home/lbw/CLODE/eval_CLODE_pkl/LOL_eval')
cache_dir.mkdir(parents=True, exist_ok=True)

# eval15 데이터 불러오기
eval_path = Path('/home/lbw/data/eval15')
eval_images = [f for f in sorted(os.listdir(eval_path / 'low')) if f.lower().endswith('.png')]

def load_eval_image(idx):
    img_name = eval_images[idx]
    lq_img = image_tensor(eval_path / 'low' / img_name)
    lq_img_224 = image_tensor(eval_path / 'low' / img_name, size=(224, 224))
    gt_img = image_tensor(eval_path / 'high' / img_name)
    
    return lq_img.to(device), gt_img.to(device), img_name, lq_img_224.to(device)

# T 값들 설정
T_values = np.linspace(2, 5, 30)
T_tensors = [torch.tensor([0, T]).float().cuda() for T in T_values]

# 이미지별 결과 저장용 딕셔너리
all_results = {}

# 이미지별 NODE 계산 및 결과 저장
for idx in tqdm(range(len(eval_images))):
    lq_img, gt_img, img_name, lq_img_224 = load_eval_image(idx)
    
    # 결과 저장용 딕셔너리
    image_results = {
        'img_name': img_name,
        'lq_img': lq_img.cpu(),
        'gt_img': gt_img.cpu(),
        'lq_img_224': lq_img_224.cpu(),
        'T_values': T_values,
        'preds': [],
        'psnrs': [],
        'ssims': []
    }
    
    # 모든 T에 대한 예측 수행
    with torch.no_grad():
        for T_tensor in T_tensors:
            pred = model(lq_img, T_tensor, inference=True)['output'][0]
            # 결과 저장 (CPU로 이동)
            image_results['preds'].append(pred.cpu())
            psnr = calculate_psnr(pred, gt_img).item()
            image_results['psnrs'].append(psnr)
            # SSIM 계산
            ssim_value = calculate_ssim(pred, gt_img)
            image_results['ssims'].append(ssim_value)
    
    # PSNR 기반 best_T 찾기
    best_T_idx = np.argmax(image_results['psnrs'])
    image_results['best_T'] = T_values[best_T_idx]
    image_results['best_T_idx'] = best_T_idx
    image_results['best_psnr'] = image_results['psnrs'][best_T_idx]
    image_results['best_ssim'] = image_results['ssims'][best_T_idx]
    
    # 결과 저장
    all_results[idx] = image_results
    
    # 메모리 관리를 위해 중간 저장
    if idx % 5 == 0 or idx == len(eval_images) - 1:
        with open(cache_dir / f'model_results_batch_{idx//5}.pkl', 'wb') as f:
            pickle.dump({k: all_results[k] for k in all_results if k//5 == idx//5}, f)
        
        # 메모리 확보
        if idx % 5 == 4:
            for k in list(all_results.keys()):
                if k//5 == (idx//5 - 1):
                    del all_results[k]

# 모든 결과의 요약 통계 저장
summary = {
    'image_count': len(eval_images),
    'T_values': T_values.tolist(),
    'best_Ts': [all_results[idx]['best_T'] for idx in range(len(eval_images)) if idx in all_results],
    'best_psnrs': [all_results[idx]['best_psnr'] for idx in range(len(eval_images)) if idx in all_results],
    'best_ssims': [all_results[idx]['best_ssim'] for idx in range(len(eval_images)) if idx in all_results],
}

# 요약 정보 저장
with open(cache_dir / 'summary.pkl', 'wb') as f:
    pickle.dump(summary, f)

print(f"모든 이미지({len(eval_images)}개)의 처리 결과를 {cache_dir}에 저장했습니다.")
print(f"이제 regressor만 변경하면서 빠르게 실험할 수 있습니다.")

In [ ]:
import torch
from tqdm import tqdm
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pickle
from torchmetrics.multimodal import CLIPImageQualityAssessment
from network.clip_regressor import TtoTRegressor

# GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 결과 저장을 위한 디렉토리 생성
results_dir = Path('/home/lbw/CLODE/result_img/CLODE_eval_results_low_feature_score2')
results_dir.mkdir(parents=True, exist_ok=True)

# 캐시된 결과 불러오기
cache_dir = Path('/home/lbw/CLODE/cached_results')
eval_path = Path('/home/lbw/data/eval15')
eval_images = [f for f in sorted(os.listdir(eval_path / 'low')) if f.lower().endswith('.png')]

# 요약 정보 불러오기
with open(cache_dir / 'summary.pkl', 'rb') as f:
    summary = pickle.load(f)

T_values = np.array(summary['T_values'])

# 하이퍼파라미터 설정
input_dim = 768
hidden_dim = 64

# 모델 초기화
regressor = Regressor(input_dim, 3, hidden_dim).to(device)
regressor.eval()
regressor.to(device)
model_path_name = (model_path / f'att_regression_{best_epoch}.pth')
print(model_path_name, input_dim)
regressor.load_state_dict(torch.load(model_path_name, map_location=device))

# CLIP 설정
prompts = ('brightness', 'noisiness', 'quality')
clip_metric = CLIPImageQualityAssessment(
    model_name_or_path="openai/clip-vit-large-patch14",
    prompts=prompts
).to(device)

clip_vision_encoder = clip_metric.model.vision_model
clip_vision_encoder.eval()
clip_vision_encoder.to(device)

clip_visual_projection = clip_metric.model.visual_projection
clip_visual_projection.eval()
clip_visual_projection.to(device)

def calculate_clip_score(pred, prompts=prompts):    
    # 이미 배치 차원이 있는지 확인하고 없으면 추가
    if len(pred.shape) == 3:
        pred = pred.unsqueeze(0)
    
    with torch.no_grad():
        # 한 번의 forward pass로 모든 프롬프트에 대한 점수를 계산
        scores = clip_metric(pred)
        encoder_feature = clip_vision_encoder(pred) 
        projection = clip_visual_projection(encoder_feature[1]) 
        
        # scores 딕셔너리를 텐서로 변환
        scores_tensor = torch.tensor([
            scores[prompts[0]].item(),
            scores[prompts[1]].item(),
            scores[prompts[2]].item()
        ], device=device).unsqueeze(0)  
        
        combined_features = torch.cat([projection, scores_tensor], dim=1)

    return combined_features

def tensor_to_numpy(tensor):
    """PyTorch 텐서를 NumPy 배열로 변환합니다."""
    # GPU -> CPU 이동 및 계산 그래프에서 분리
    img = tensor.detach().cpu().numpy()
    
    # 이미지 형식 변환 (C,H,W) -> (H,W,C)
    if img.ndim == 3 and img.shape[0] == 3:
        img = np.transpose(img, (1, 2, 0))
    
    # 값 범위 조정 [0, 1]
    img = np.clip(img, 0, 1)
    
    return img

# Regressor 평가용 변수
pred_Ts = []
pred_psnrs = []
pred_ssims = []

# 이미지별 결과 불러오고 regressor 평가
for idx in tqdm(range(len(eval_images))):
    # 이미지의 결과가 어떤 배치 파일에 있는지 계산
    batch_idx = idx // 5
    
    # 배치 파일 로드
    with open(cache_dir / f'model_results_batch_{batch_idx}.pkl', 'rb') as f:
        batch_results = pickle.load(f)
    
    # 현재 이미지 결과
    img_results = batch_results[idx]
    
    # 필요한 데이터 추출
    lq_img_224 = img_results['lq_img_224'].to(device)
    best_T = img_results['best_T']
    best_psnr = img_results['best_psnr']
    best_ssim = img_results['best_ssim']
    psnrs = img_results['psnrs']
    ssims = img_results['ssims']
    img_name = img_results['img_name']
    
    # CLIP 점수 계산 및 행렬 구성
    clip_feature = calculate_clip_score(lq_img_224)

    # Regressor로 T 예측
    with torch.no_grad():
        pred_T = regressor(clip_feature)
        pred_T = pred_T.item()
    
    # 가장 가까운 T 값 찾기
    pred_T_idx = np.argmin(np.abs(T_values - pred_T))
    
    # 결과 저장
    pred_Ts.append(pred_T)
    pred_psnrs.append(psnrs[pred_T_idx])
    pred_ssims.append(ssims[pred_T_idx])
    
    # 시각화 (필요한 경우)
    plt.figure(figsize=(20, 15))
    
    # 원본 저화질 이미지
    plt.subplot(2, 2, 1)
    plt.imshow(tensor_to_numpy(img_results['lq_img'][0].cpu()))
    plt.title(f'Low Quality Image: {img_name}')
    plt.axis('off')
    
    # 원본 고화질 이미지
    plt.subplot(2, 2, 2)
    plt.imshow(tensor_to_numpy(img_results['gt_img'][0].cpu()))
    plt.title('Ground Truth')
    plt.axis('off')
    
    # PSNR 기반 best_T 이미지
    plt.subplot(2, 2, 3)
    plt.imshow(tensor_to_numpy(img_results['preds'][img_results['best_T_idx']].cpu()))
    plt.title(f'NODE Best T={best_T:.2f}, PSNR={best_psnr:.2f}dB, SSIM={best_ssim:.4f}')
    plt.axis('off')
    
    # Regressor 예측 T 이미지
    plt.subplot(2, 2, 4)
    plt.imshow(tensor_to_numpy(img_results['preds'][pred_T_idx].cpu()))
    plt.title(f'Regressor Pred T={pred_T:.2f}, PSNR={psnrs[pred_T_idx]:.2f}dB, SSIM={ssims[pred_T_idx]:.4f}')
    plt.axis('off')
    
    # 결과 저장
    plt.tight_layout()
    plt.savefig(results_dir / f'{img_name.split(".")[0]}_compare.png')
    plt.close()

# 통계 정보 계산 (전체 이미지에 대해)
best_Ts = summary['best_Ts']
best_psnrs = summary['best_psnrs']
best_ssims = summary['best_ssims']

print("모든 이미지 처리 완료!")

# best_Ts와 pred_Ts를 하나의 플롯에 시각화
plt.figure(figsize=(10, 6))
plt.plot(best_Ts, label='Best Ts (PSNR-based)', marker='o')
plt.plot(pred_Ts, label='Predicted Ts (Regressor-based)', marker='x')
plt.xlabel('Image Index')
plt.ylabel('T Values')
plt.title('Comparison of Best Ts and Predicted Ts')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(results_dir / 'best_vs_pred_Ts.png')
plt.show()

# 결과 비교
print(f"Best T PSNR 평균: {np.mean(best_psnrs):.2f}dB")
print(f"Predicted T PSNR 평균: {np.mean(pred_psnrs):.2f}dB")
print(f"Best T SSIM 평균: {np.mean(best_ssims):.4f}")
print(f"Predicted T SSIM 평균: {np.mean(pred_ssims):.4f}")